## Basic test of neural nets

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as pyplot
import numpy as np

# configure device to use gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# hyperparameters
learning_rate = 0.01
num_epochs = 1000
input_size = 1
hidden_layers_sizes = [8, 4, 2]
out_classes = 1
# batch_size = 5


x_train = torch.tensor([[1], [2], [3], [4], [5], [6], [7], [8], [9], [10]], dtype=torch.float32).to(device)
y_train = torch.tensor([[8], [11], [14], [17], [20], [23], [26], [29], [32], [35]], dtype=torch.float32).to(device)

x_test = torch.tensor([[15]], dtype=torch.float32).to(device)

class NeuralNet(nn.Module):
  def __init__(self, input_size, hidden_layers_sizes, out_classes):
    super(NeuralNet, self).__init__()

    layers = []
    current_input_size = input_size

    for hidden_size in hidden_layers_sizes:
      layers.append(nn.Linear(current_input_size, hidden_size))
      layers.append(nn.ReLU())
      current_input_size = hidden_size

    layers.append(nn.Linear(current_input_size, out_classes))

    self.network = nn.Sequential(*layers)

  def forward(self, x):
    return self.network(x)


model = NeuralNet(input_size, hidden_layers_sizes, out_classes).to(device)

# loss function and optimizer function
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# train the model
for epoch in range(num_epochs):
  out_data = model(x_train)

  l = criterion(out_data, y_train)
  l.backward()

  optimizer.step()
  optimizer.zero_grad()

print(model(x_test).item())





50.06367874145508


In [18]:
# from google.colab import drive
# drive.mount('/content/drive')
# !ls
%cd /content/drive/MyDrive/Colab Notebooks

!git clone https://github.com/Amit-adh/AI-Algorithm-Analysis.git
!mv bias_analysis.ipynb AI-Algorithm-Analysis/
%cd AI-Algorithm-Analysis
!git add bias_analysis.ipynb # Adds all files in the current directory
!git commit -m "Initial commit from Colab notebook"
!git push -u origin main

Mounted at /content/drive
drive  sample_data


In [26]:
!pwd

/content/drive/MyDrive/Colab Notebooks


In [ ]:
!git config --global user.email "itsman38@gmail.com"
!git config --global user.name "Amit-adh"

/content
